# Joining the data together for Stata analysis

1. Load Graph data

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import json

pd.set_option("display.max_columns", None)

Graph data is problematic - fix this as there are issues with sparseness, use a different graph format later

Load the data from the 1.5 notebook

In [29]:
boardex_graph_stats_df = pd.read_csv("final_data/boardex_graph_stats_gvkey.csv", index_col= 0)

In [32]:
boardex_graph_stats_df.head()

,permco,gvkey,companyid,score,preferred,duplicate,boardid,boardname,year,local_clustering_coef,degree_centrality,betweenness_centrality,graph_density
0,216,1602,2068.0,1.0,1.0,0.0,2068.0,AMGEN INC,2022,0.428352,0.001668,9.543488e-04,0.003216
1,216,1602,2068.0,1.0,1.0,0.0,2068.0,AMGEN INC,2023,0.156650,0.002396,1.115499e-03,0.004495
2,682,18738,28834.0,1.0,1.0,0.0,28834.0,DOMINARI HOLDINGS INC (AIkido Pharma Inc prior...,2022,0.000000,0.000852,6.180685e-07,0.003216
3,682,18738,28834.0,1.0,1.0,0.0,28834.0,DOMINARI HOLDINGS INC (AIkido Pharma Inc prior...,2023,0.000000,0.000945,0.000000e+00,0.004495
4,887,12713,2179515.0,6.0,1.0,1.0,2179515.0,ABEONA THERAPEUTICS INC (PlasmaTech Biopharmac...,2022,0.000000,0.000591,0.000000e+00,0.003216


In [33]:
boardex_graph_stats_df = boardex_graph_stats_df[[
    "gvkey", "year", "local_clustering_coef", "degree_centrality", "betweenness_centrality", "graph_density"
]]

In [34]:
boardex_graph_stats_df.head()

,gvkey,year,local_clustering_coef,degree_centrality,betweenness_centrality,graph_density
0,1602,2022,0.428352,0.001668,9.543488e-04,0.003216
1,1602,2023,0.156650,0.002396,1.115499e-03,0.004495
2,18738,2022,0.000000,0.000852,6.180685e-07,0.003216
3,18738,2023,0.000000,0.000945,0.000000e+00,0.004495
4,12713,2022,0.000000,0.000591,0.000000e+00,0.003216


2. Load the Text topic and sentiment data

In [2]:
# load the sentiment data
sentiment_10k_results_df = pd.read_json("../2.Initial_Graph_Building/annual_report_data/all_companies_report_annual_report_text_2022_2024_with_sentiments.json")


In [3]:
# load the topic data

topic_model_10k_results_df = pd.read_json("all_companies_metadata_2022_2024_text_1A_cleaned_w_topic_dist.json")

In [4]:
topic_model_10k_results_df.columns

Index(['id', 'accessionNo', 'cik', 'ticker', 'companyName', 'companyNameLong',
       'formType', 'description', 'filedAt', 'linkToTxt', 'linkToHtml',
       'linkToXbrl', 'linkToFilingDetails', 'entities', 'documentFormatFiles',
       'dataFiles', 'seriesAndClassesContractsInformation', 'periodOfReport',
       'Text_1', 'Text_1A', 'effectivenessDate', 'Text_1A_data_cleaned',
       'topic_distribution'],
      dtype='object')

In [5]:
sentiment_10k_results_df.columns

Index(['id', 'accessionNo', 'cik', 'ticker', 'companyName', 'companyNameLong',
       'formType', 'description', 'filedAt', 'linkToTxt', 'linkToHtml',
       'linkToXbrl', 'linkToFilingDetails', 'entities', 'documentFormatFiles',
       'dataFiles', 'seriesAndClassesContractsInformation', 'periodOfReport',
       'Text_1', 'Text_1A', 'effectivenessDate', 'risk_sentiment',
       'business_overview_sentiment'],
      dtype='object')

In [6]:
sentiment_10k_results_df.head()

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,Text_1,Text_1A,effectivenessDate,risk_sentiment,business_overview_sentiment
0,c192751c88f50d8ca603ab72c5840583,0000318154-24-000011,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2024-02-14T16:23:32-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '17', 'description': 'XBRL TAXON...",[],2023-12-31,Item 1. BUSINESS ##TABLE_END\n\nAmgen Inc. (i...,Item 1A. RISK FACTORS ##TABLE_END\n\nThis rep...,None,0,-1
1,3f8f2f776c43180b8573405b00605c15,0000318154-23-000017,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-09T16:26:31-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '11', 'description': 'XBRL TAXON...",[],2022-12-31,Item 1. BUSINESS ##TABLE_END\n\nAmgen Inc. (i...,Item 1A. RISK FACTORS ##TABLE_END\n\nThis rep...,None,-1,-1
2,1d205d8c366bf6e51746f33967a1141d,0000318154-22-000010,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-16T16:39:53-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '13', 'description': 'XBRL TAXON...",[],2021-12-31,Item 1. BUSINESS ##TABLE_END\n\nAmgen Inc. (i...,Item 1A. RISK FACTORS ##TABLE_END\n\nThis rep...,None,0,1
3,358cf2de3309caed349f7d4505bf6d49,0001628280-24-005397,9326,BCPC,BALCHEM CORP,BALCHEM CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2024-02-16T17:26:27-05:00,https://www.sec.gov/Archives/edgar/data/9326/0...,https://www.sec.gov/Archives/edgar/data/9326/0...,,https://www.sec.gov/Archives/edgar/data/9326/0...,"[{'companyName': 'BALCHEM CORP (Filer)', 'cik'...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '14', 'description': 'XBRL TAXON...",[],2023-12-31,"Item 1. Business (All amounts in thousands, e...",Item 1A. Risk Factors \n\nWe discuss our expe...,None,-1,-1
4,1941cf293846a69b64943f8b2b3ff87f,0001628280-23-005074,9326,BCPC,BALCHEM CORP,BALCHEM CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-24T16:43:02-05:00,https://www.sec.gov/Archives/edgar/data/9326/0...,https://www.sec.gov/Archives/edgar/data/9326/0...,,https://www.sec.gov/Archives/edgar/data/9326/0...,"[{'companyName': 'BALCHEM CORP (Filer)', 'cik'...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '12', 'description': 'XBRL TAXON...",[],2022-12-31,"Item 1. Business (All amounts in thousands, e...",Item 1A. Risk Factors \n\nWe discuss our expe...,None,-1,-1


In [7]:
# sentiment_10k_results_df.head()

# change to period of report later - that is the key field after looking at this

In [8]:
sentiment_10k_results_df.head()["description"][0]

'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]'

In [9]:
pd.to_datetime(sentiment_10k_results_df["periodOfReport"]).apply(lambda x : x.year)

0       2023
1       2022
2       2021
3       2023
4       2022
        ... 
2368    2023
2369    2022
2370    2021
2371    2023
2372    2022
Name: periodOfReport, Length: 2373, dtype: int64

In [10]:
annual_report_features_df = sentiment_10k_results_df.copy()

In [11]:
annual_report_features_df.drop(columns=['Text_1', 'Text_1A', ])

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate,risk_sentiment,business_overview_sentiment
0,c192751c88f50d8ca603ab72c5840583,0000318154-24-000011,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2024-02-14T16:23:32-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '17', 'description': 'XBRL TAXON...",[],2023-12-31,None,0,-1
1,3f8f2f776c43180b8573405b00605c15,0000318154-23-000017,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-09T16:26:31-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '11', 'description': 'XBRL TAXON...",[],2022-12-31,None,-1,-1
2,1d205d8c366bf6e51746f33967a1141d,0000318154-22-000010,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-16T16:39:53-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '13', 'description': 'XBRL TAXON...",[],2021-12-31,None,0,1
3,358cf2de3309caed349f7d4505bf6d49,0001628280-24-005397,9326,BCPC,BALCHEM CORP,BALCHEM CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2024-02-16T17:26:27-05:00,https://www.sec.gov/Archives/edgar/data/9326/0...,https://www.sec.gov/Archives/edgar/data/9326/0...,,https://www.sec.gov/Archives/edgar/data/9326/0...,"[{'companyName': 'BALCHEM CORP (Filer)', 'cik'...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '14', 'description': 'XBRL TAXON...",[],2023-12-31,None,-1,-1
4,1941cf293846a69b64943f8b2b3ff87f,0001628280-23-005074,9326,BCPC,BALCHEM CORP,BALCHEM CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-24T16:43:02-05:00,https://www.sec.gov/Archives/edgar/data/9326/0...,https://www.sec.gov/Archives/edgar/data/9326/0...,,https://www.sec.gov/Archives/edgar/data/9326/0...,"[{'companyName': 'BALCHEM CORP (Filer)', 'cik'...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '12', 'description': 'XBRL TAXON...",[],2022-12-31,None,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,437a263932014ad06bcdbde870c7ae88,0001493152-24-009769,1857044,INDP,"Indaptus Therapeutics, Inc.","Indaptus Therapeutics, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2024-03-13T08:02:27-04:00,https://www.sec.gov/Archives/edgar/data/185704...,https://www.sec.gov/Archives/edgar/data/185704...,,https://www.sec.gov/Archives/edgar/data/185704...,"[{'companyName': 'Indaptus Therapeutics, Inc. ...","[{'sequence': '1', 'documentUrl': 'https://www...","[{'sequence': '16', 'description': 'XBRL SCHEM...",[],2023-12-31,None,0,-1
2369,8751e66576529b1cda4f7faf672dbdc1,0001493152-23-008010,1857044,INDP,"Indaptus Therapeutics, Inc.","Indaptus Therapeutics, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-03-17T08:05:56-04:00,https://www.sec.gov/Archives/edgar/data/185704...,https://www.sec.gov/Archives/edgar/data/185704...,,https://www.sec.gov/Archives/edgar/data/185704...,"[{'companyName': 'Indaptus Therapeutics, Inc. ...","[{'sequence': '1', 'documentUrl': 'https://www...","[{'sequence': '18', 'description': 'XBRL SCHE

In [12]:
annual_report_features_df["report_year"] = pd.to_datetime(annual_report_features_df["periodOfReport"]).apply(lambda x : x.year)


In [14]:
# get topics
annual_report_features_df["risk_topic_0_allocation"] = topic_model_10k_results_df["topic_distribution"].apply(lambda x: x[0][-1])
annual_report_features_df["risk_topic_1_allocation"] = topic_model_10k_results_df["topic_distribution"].apply(lambda x: x[1][-1])
annual_report_features_df["risk_topic_2_allocation"] = topic_model_10k_results_df["topic_distribution"].apply(lambda x: x[2][-1])
annual_report_features_df["risk_topic_3_allocation"] = topic_model_10k_results_df["topic_distribution"].apply(lambda x: x[3][-1])

In [16]:
annual_report_features_df.head() # example for distribution approach - check result

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,Text_1,Text_1A,effectivenessDate,risk_sentiment,business_overview_sentiment,report_year,risk_topic_0_allocation,risk_topic_1_allocation,risk_topic_2_allocation,risk_topic_3_allocation
0,c192751c88f50d8ca603ab72c5840583,0000318154-24-000011,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2024-02-14T16:23:32-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '17', 'description': 'XBRL TAXON...",[],2023-12-31,Item 1. BUSINESS ##TABLE_END\n\nAmgen Inc. (i...,Item 1A. RISK FACTORS ##TABLE_END\n\nThis rep...,None,0,-1,2023,0.087753,0.905426,0.002893,0.003929
1,3f8f2f776c43180b8573405b00605c15,0000318154-23-000017,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-09T16:26:31-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '11', 'description': 'XBRL TAXON...",[],2022-12-31,Item 1. BUSINESS ##TABLE_END\n\nAmgen Inc. (i...,Item 1A. RISK FACTORS ##TABLE_END\n\nThis rep...,None,-1,-1,2022,0.064385,0.926407,0.004403,0.004806
2,1d205d8c366bf6e51746f33967a1141d,0000318154-22-000010,318154,AMGN,AMGEN INC,AMGEN INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-16T16:39:53-05:00,https://www.sec.gov/Archives/edgar/data/318154...,https://www.sec.gov/Archives/edgar/data/318154...,,https://www.sec.gov/Archives/edgar/data/318154...,"[{'companyName': 'AMGEN INC (Filer)', 'cik': '...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '13', 'description': 'XBRL TAXON...",[],2021-12-31,Item 1. BUSINESS ##TABLE_END\n\nAmgen Inc. (i...,Item 1A. RISK FACTORS ##TABLE_END\n\nThis rep...,None,0,1,2021,0.085579,0.904845,0.005602,0.003974
3,358cf2de3309caed349f7d4505bf6d49,0001628280-24-005397,9326,BCPC,BALCHEM CORP,BALCHEM CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2024-02-16T17:26:27-05:00,https://www.sec.gov/Archives/edgar/data/9326/0...,https://www.sec.gov/Archives/edgar/data/9326/0...,,https://www.sec.gov/Archives/edgar/data/9326/0...,"[{'companyName': 'BALCHEM CORP (Filer)', 'cik'...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '14', 'description': 'XBRL TAXON...",[],2023-12-31,"Item 1. Business (All amounts in thousands, e...",Item 1A. Risk Factors \n\nWe discuss our expe...,None,-1,-1,2023,0.616200,0.383668,0.000066,0.000066
4,1941cf293846a69b64943f8b2b3ff87f,0001628280-23-005074,9326,BCPC,BALCHEM CORP,BALCHEM CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-24T16:43:02-05:00,https://www.sec.gov/Archives/edgar/data/9326/0...,https://www.sec.gov/Archives/edgar/data/9326/0...,,https://www.sec.gov/Archives/edgar/data/9326/0...,"[{'companyName': 'BALCHEM CORP (Filer)', 'cik'...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '12', 'description': 'XBRL TAXON...",[],2022-12-31,"Item 1. Business (All amounts in thousands, e...",Item 1A. Risk Factors \n\nWe discuss our expe...,None,-1,-1,2022,0.626234,0.373639,0.000064,0.000064


In [19]:
annual_report_key_features = annual_report_features_df[["cik", "report_year", 
                           "risk_sentiment", "business_overview_sentiment",
                           "risk_topic_0_allocation", "risk_topic_1_allocation", 
                           "risk_topic_2_allocation", "risk_topic_3_allocation"]]

In [21]:
annual_report_key_features.to_json("final_data/annual_reports_combined_features_2021_2024.json")

In [20]:
annual_report_key_features.head()

,cik,report_year,risk_sentiment,business_overview_sentiment,risk_topic_0_allocation,risk_topic_1_allocation,risk_topic_2_allocation,risk_topic_3_allocation
0,318154,2023,0,-1,0.087753,0.905426,0.002893,0.003929
1,318154,2022,-1,-1,0.064385,0.926407,0.004403,0.004806
2,318154,2021,0,1,0.085579,0.904845,0.005602,0.003974
3,9326,2023,-1,-1,0.616200,0.383668,0.000066,0.000066
4,9326,2022,-1,-1,0.626234,0.373639,0.000064,0.000064


#### Link GVKEY using the data dictionary

Same as boardex principles, but with another table, and then combine these together!!

2.Initial_Graph_Building/sec_gvkey_cik_us_pharma_2022_2024.csv

Match compustat with boardex and annual report date here.

In [23]:
# get year company details, 

ar_key_df = pd.read_csv("../2.Initial_Graph_Building/sec_gvkey_cik_us_pharma_2022_2024.csv", index_col = 0)

ar_key_df.head()

,gvkey,conm,datadate1,datadate2,cik,source,coname,fndate,lndate,n10k,n10k_nt,n10k_a,n10q,n10q_nt,n10q_a,ndef,n8k,ntot,flag
0,1602,AMGEN INC,1983-03-31,2023-12-31,318154.0,COMPN,AMGEN INC,1994-03-28,2024-06-04,32.0,0.0,3.0,95.0,0.0,4.0,92.0,308.0,2936.0,3.0
1,1979,BALCHEM CORP -CL B,1974-12-31,2023-12-31,9326.0,COMPN,BALCHEM CORP,1995-08-11,2024-05-24,31.0,0.0,2.0,91.0,0.0,5.0,38.0,194.0,1148.0,3.0
2,2086,BAXTER INTERNATIONAL INC,1950-12-31,2023-12-31,10456.0,COMPN,BAXTER INTERNATIONAL INC,1994-03-21,2024-06-05,34.0,1.0,3.0,97.0,1.0,5.0,52.0,276.0,2321.0,3.0
3,2403,BRISTOL-MYERS SQUIBB CO,1950-12-31,2023-12-31,14272.0,COMPN,BRISTOL MYERS SQUIBB CO,1994-03-15,2024-05-09,33.0,0.0,3.0,98.0,1.0,6.0,64.0,373.0,3304.0,3.0
4,2990,IMUNON INC,1983-09-30,2023-12-31,749647.0,COMPN,"IMUNON, INC.",1996-08-07,2024-05-22,38.0,1.0,9.0,96.0,3.0,9.0,41.0,300.0,1263.0,3.0


In [25]:
annual_report_key_features_final_df = pd.merge(ar_key_df[["gvkey", "cik"]], annual_report_key_features, how = "inner", on = "cik")

In [26]:
annual_report_key_features_final_df.to_csv("../4. Feature_Generation/final_data/sec_gvkey_ar_report_features.csv")

### Finally link with the compustat data

First review graph and company level data

In [35]:
boardex_graph_stats_df.head()

,gvkey,year,local_clustering_coef,degree_centrality,betweenness_centrality,graph_density
0,1602,2022,0.428352,0.001668,9.543488e-04,0.003216
1,1602,2023,0.156650,0.002396,1.115499e-03,0.004495
2,18738,2022,0.000000,0.000852,6.180685e-07,0.003216
3,18738,2023,0.000000,0.000945,0.000000e+00,0.004495
4,12713,2022,0.000000,0.000591,0.000000e+00,0.003216


In [36]:
annual_report_key_features_final_df.head()

,gvkey,cik,report_year,risk_sentiment,business_overview_sentiment,risk_topic_0_allocation,risk_topic_1_allocation,risk_topic_2_allocation,risk_topic_3_allocation
0,1602,318154.0,2023,0,-1,0.087753,0.905426,0.002893,0.003929
1,1602,318154.0,2022,-1,-1,0.064385,0.926407,0.004403,0.004806
2,1602,318154.0,2021,0,1,0.085579,0.904845,0.005602,0.003974
3,1979,9326.0,2023,-1,-1,0.616200,0.383668,0.000066,0.000066
4,1979,9326.0,2022,-1,-1,0.626234,0.373639,0.000064,0.000064


Start merging the results with compustat, just merge on year and company id (gvkey - compustat id)

In [38]:
graph_ar_features = pd.merge(boardex_graph_stats_df, annual_report_key_features_final_df, 
                             how = "inner", left_on= ["gvkey", "year"], right_on=["gvkey", "report_year"])
graph_ar_features.head()

,gvkey,year,local_clustering_coef,degree_centrality,betweenness_centrality,graph_density,cik,report_year,risk_sentiment,business_overview_sentiment,risk_topic_0_allocation,risk_topic_1_allocation,risk_topic_2_allocation,risk_topic_3_allocation
0,1602,2022,0.428352,0.001668,9.543488e-04,0.003216,318154.0,2022,-1,-1,0.064385,0.926407,0.004403,0.004806
1,1602,2023,0.156650,0.002396,1.115499e-03,0.004495,318154.0,2023,0,-1,0.087753,0.905426,0.002893,0.003929
2,18738,2022,0.000000,0.000852,6.180685e-07,0.003216,12239.0,2022,-1,-1,0.971470,0.008822,0.019662,0.000046
3,18738,2023,0.000000,0.000945,0.000000e+00,0.004495,12239.0,2023,-1,-1,0.959382,0.009551,0.031023,0.000044
4,12713,2022,0.000000,0.000591,0.000000e+00,0.003216,318306.0,2022,-1,-1,0.250000,0.250000,0.250000,0.250000


In [50]:
graph_ar_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1177 entries, 0 to 1176
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gvkey                        1177 non-null   int64  
 1   year                         1177 non-null   int64  
 2   local_clustering_coef        1177 non-null   float64
 3   degree_centrality            1177 non-null   float64
 4   betweenness_centrality       1177 non-null   float64
 5   graph_density                1177 non-null   float64
 6   cik                          1177 non-null   float64
 7   report_year                  1177 non-null   int64  
 8   risk_sentiment               1177 non-null   int64  
 9   business_overview_sentiment  1177 non-null   int64  
 10  risk_topic_0_allocation      1177 non-null   float64
 11  risk_topic_1_allocation      1177 non-null   float64
 12  risk_topic_2_allocation      1177 non-null   float64
 13  risk_topic_3_alloc

In [57]:
# get compustat data

compustat_workflow_df = pd.read_csv("../2.Initial_Graph_Building/compustat_pharma_2021_2024_manual.csv", low_memory=False)

In [58]:
compustat_workflow_df.columns[:20]

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'cusip', 'conm', 'acctchg', 'acctstd', 'acqmeth', 'adrr', 'ajex',
       'ajp', 'bspr', 'compst', 'curcd', 'curncd'],
      dtype='object')

In [59]:
compustat_workflow_df["gvkey"]

0         1602
1         1602
2         1602
3         2403
4         2403
         ...  
3046    351038
3047    351038
3048    353444
3049    353444
3050    353444
Name: gvkey, Length: 3051, dtype: int64

In [60]:
compustat_workflow_df["fyear"] = compustat_workflow_df["fyear"].astype(int)

In [65]:
final_merged_data_df = pd.merge(compustat_workflow_df, graph_ar_features, 
         how = "inner", left_on=["gvkey","fyear"], 
         right_on=["gvkey","year"])

In [66]:
final_merged_data_df.to_csv("final_data/computat_boardex_10k_all_merged.csv")

In [67]:
final_merged_data_df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,acctchg,acctstd,acqmeth,adrr,ajex,ajp,bspr,compst,curcd,curncd,currtr,curuscn,final,fyr,ismod,ltcm,ogm,pddur,scf,src,stalt,udpl,upd,apdedate,fdate,pdate,acchg,acco,accrt,acdo,aco,acodo,acominc,acox,acoxar,acqao,acqcshi,acqgdwl,acqic,acqintan,acqinvt,acqlntal,acqniintc,acqppe,acqsc,act,adpac,aedi,afudcc,afudci,aldo,am,amc,amdc,amgw,ano,ao,aocidergl,aociother,aocipen,aocisecgl,aodo,aol2,aoloch,aox,ap,apalch,apb,apc,apofs,aqa,aqc,aqd,aqeps,aqi,aqp,aqpl1,aqs,arb,arc,arce,arced,arceeps,artfs,at,aul3,autxr,balr,banlr,bast,bastr,batr,bcef,bclr,bcltbl,bcnlr,bcrbl,bct,bctbl,bctr,billexce,bkvlps,bltbl,ca,capr1,capr2,capr3,caps,capsft,capx,capxv,cb,cbi,cdpac,cdvc,ceiexbill,ceq,ceql,ceqt,cfbd,cfere,cfo,cfpdo,cga,cgri,cgti,cgui,ch,che,chech,chs,ci,cibegni,cicurr,cidergl,cimii,ciother,cipen,cisecgl,citotal,cld2,cld3,cld4,cld5,clfc,clfx,clg,clis,cll,cllc,clo,clrll,clt,cmp,cnltbl,cogs,cpcbl,cpdoi,cpnli,cppbl,cprei,crv,crvnli,cshfd,cshi,csho,cshpri,cshr,cshrc,cshrp,cshrso,cshrt,cshrw,cstk,cstkcv,cstke,dbi,dc,dclo,dcom,dcpstk,dcs,dcvsr,dcvsub,dcvt,dd,dd1,dd2,dd3,dd4,dd5,depc,derac,deralt,derhedgl,derlc,derllt,dfpac,dfs,dfxa,diladj,dilavx,dlc,dlcch,dltis,dlto,dltp,dltr,dltsub,dltt,dm,dn,do,donr,dp,dpacb,dpacc,dpacli,dpacls,dpacme,dpacnr,dpaco,dpacre,dpact,dpc,dpdc,dpltb,dpret,dpsc,dpstb,dptb,dptc,dptic,dpvieb,dpvio,dpvir,drc,drci,drlt,ds,dt,dtea,dted,dteeps,dtep,dudd,dv,dvc,dvdnp,dvintf,dvp,dvpa,dvpd,dvpdp,dvpibb,dvrpiv,dvrre,dvsco,dvt,dxd2,dxd3,dxd4,dxd5,ea,ebit,ebitda,eiea,emol,emp,epsfi,epsfx,epspi,epspx,esopct,esopdlt,esopnr,esopr,esopt,esub,esubc,excadj,exre,fatb,fatc,fatd,fate,fatl,fatn,fato,fatp,fca,fdfr,fea,fel,ffo,ffs,fiao,finaco,finao,fincf,finch,findlc,findlt,finivst,finlco,finlto,finnp,finrecc,finreclt,finrev,finxint,finxopr,fopo,fopox,fopt,fsrco,fsrct,fuseo,fuset,gbbl,gdwl,gdwlam,gdwlia,gdwlid,gdwlieps,gdwlip,geqrv,gla,glcea,glced,glceeps,glcep,gld,gleps,gliv,glp,govgr,govtown,gp,gphbl,gplbl,gpobl,gprbl,gptbl,gwo,hedgegl,iaeq,iaeqci,iaeqmi,iafici,iafxi,iafxmi,iali,ialoi,ialti,iamli,iaoi,iapli,iarei,iasci,iasmi,iassi,iasti,iatci,iati,iatmi,iaui,ib,ibadj,ibbl,ibc,ibcom,ibki,ibmii,icapt,idiis,idilb,idilc,idis,idist,idit,idits,iire,initb,intan,intano,intc,intpn,invch,invfg,invo,invofs,invreh,invrei,invres,invrm,invt,invwip,iobd,ioi,iore,ip,ipabl,ipc,iphbl,iplbl,ipobl,iptbl,ipti,ipv,irei,irent,irii,irli,irnli,irsi,iseq,iseqc,iseqm,isfi,isfxc,isfxm,isgr,isgt,isgu,islg,islgc,islgm,islt,isng,isngc,isngm,isotc,isoth,isotm,issc,issm,issu,ist,istc,istm,isut,itcb,itcc,itci,ivaco,ivaeq,ivao,ivch,ivgod,ivi,ivncf,ivpt,ivst,ivstch,lcabg,lcacl,lcacr,lcag,lcal,lcalt,lcam,lcao,lcast,lcat,lco,lcox,lcoxar,lcoxdr,lct,lcuacu,li,lif,lifr,lifrp,lloml,lloo,llot,llrci,llrcr,llwoci,llwocr,lno,lo,lol2,loxdr,lqpl1,lrv,ls,lse,lst,lt,lul3,mib,mibn,mibt,mii,mrc1,mrc2,mrc3,mrc4,mrc5,mrct,mrcta,msa,msvrv,mtl,nat,nco,nfsr,ni,niadj,nieci,niint,niintpfc,niintpfp,niit,nim,nio,nipfc,nipfp,nit,nits,nopi,nopio,np,npanl,npaore,nparl,npat,nrtxt,nrtxtd,nrtxteps,oancf,ob,oiadp,oibdp,opeps,opili,opincar,opini,opioi,opiri,opiti,oprepsx,optca,optdr,optex,optexd,optfvgr,optgr,optlife,optosby,optosey,optprcby,optprcca,optprcex,optprcey,optprcgr,optprcwa,optrfr,optvol,palr,panlr,patr,pcl,pclr,pcnlr,pctr,pdvc,pi,pidom,pifo,pll,pltbl,pnca,pncad,pncaeps,pncia,pncid,pncieps,pncip,pncwia,pncwid,pncwieps,pncwip,pnlbl,pnli,pnrsho,pobl,ppcbl,ppegt,ppenb,ppenc,ppenli,ppenls,ppenme,ppennr,ppeno,ppent,ppevbb,ppeveb,ppevo,ppevr,pppabl,ppphbl,pppobl,ppptbl,prc,prca,prcad,prcaeps,prebl,pri,prodv,prsho,prstkc,prstkcc,prstkpc,prvt,pstk,pstkc,pstkl,pstkn,pstkr,pstkrv,ptbl,ptran,pvcl,pvo,pvon,pvpl,pvt,pwoi,radp,ragr,rari,rati,rca,rcd,rceps,rcl,rcp,rdip,rdipa,rdipd,rdipeps,rdp,re,rea,reajo,recch,recco,recd,rect,recta,rectr,recub,ret,reuna,reunr,revt,ris,rll,rlo,rlp,rlri,rlt,rmum,rpag,rra,rrd,rreps,rrp,rstche,rstchelt,rvbci,rvbpi,rvbti,rvdo,rvdt,rveqt,rvlrv,rvno,rvnt,rvri,rvsi,rvti,rvtxr,rvupi,rvutx,saa,sal,sale,salepfc,salepfp,sbdc,sc,sc

In [68]:
final_merged_data_df.shape

(1136, 994)

Around 1000 rows so it works fine.

In [69]:
final_merged_data_df.fyear.value_counts()

fyear
2022    653
2023    483
Name: count, dtype: int64

In [ ]:
# 2 years of data which is fine